In [2]:
from app.utils import create_pad_sequences, clean_text, load_model, load_tokenizer, BODY_COL_NAME
from typing import Dict, Any
from deep_translator import GoogleTranslator
import pandas as pd

ROOT_DIR = '../../'

In [3]:
model_by_name: Dict[str, Dict[str, Any]] = {
    'phishing': {
        'threshold': 0.985,
    },
    'fraud': {
        'threshold': 0.90,
    },
    'spam': {
        'threshold': 0.98,
    },
}

for name in model_by_name.keys():
    model_by_name[name]['model'] = load_model(f'{ROOT_DIR}models/{name}')
    model_by_name[name]['tokenizer'] = load_tokenizer(f'{ROOT_DIR}tokenizers/{name}')

DEBUG:h5py._conv:Creating converter from 3 to 5


Модель загружена!


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 10, 128)        │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,379,075 (5.26 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 10, 128)        │     1,280,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,379,075 (5.26 MB)

 Trainable params: 1,379,073 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
def predict(message: str):
    message = clean_text(message)
    df = pd.DataFrame([message], columns=[BODY_COL_NAME])

    for name, data in model_by_name.items():
        tokenizer = data['tokenizer']
        model = data['model']
        message_pad = create_pad_sequences(tokenizer, df[BODY_COL_NAME])
        prediction = model.predict(message_pad)
        prediction = prediction[0][0]
        print(name, prediction, f'Trigger: {prediction > data["threshold"]}')

In [5]:
import sys

message = input("Enter eMAIL message: ")

if not message:
    sys.exit(1)

translator = GoogleTranslator(source='auto', target='en')
message = translator.translate(message)
print(message)
predict(message)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): translate.google.com:443
DEBUG:urllib3.connectionpool:https://translate.google.com:443 "GET /m?tl=en&sl=auto&q=%D0%9F%D1%80%D0%B8%D0%B2%D0%B5%D1%82%2C+%3D%D0%B8%D1%81%D0%BF%D1%83%D0%B3%D0%B0%D0%BB%D1%81%D1%8F%3F+%D0%AF+%D1%81+%D0%BF%D0%BE%D0%B4%D0%B0%D1%80%D0%BA%D0%B0%D0%BC%D0%B8%21 HTTP/11" 200 None


Hi, =are you scared? I have gifts!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
phishing 0.884911 Trigger: False
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
fraud 0.003031449 Trigger: False
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
spam 0.8149748 Trigger: False
